<a href="https://colab.research.google.com/github/kyochanpy/Google_Smartphone_Decimeter_Challenge/blob/main/note_book/submission_34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna > /dev/null
!pip install pyproj > /dev/null
!pip install simdkalman > /dev/null
    
import os
from glob import glob
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns

import optuna
import plotly
import plotly.express as px
import pyproj
from pathlib import Path
from pyproj import Proj, transform
from tqdm.notebook import tqdm
import simdkalman

In [16]:
path = Path("/content/drive/MyDrive/GSDC")
#test_base = pd.read_csv(path / "baseline_locations_test.csv")
test_base = pd.read_csv("/content/drive/MyDrive/GSDC/test_predict_next_point_SJC_2.csv")

sub = pd.read_csv(path / "sample_submission.csv")

truths = (path / "train").rglob("ground_truth.csv")

df_list = []
cols = ["collectionName", "phoneName", "millisSinceGpsEpoch", "latDeg", "lngDeg"]

for t in tqdm(truths, total=73):
    df_phone = pd.read_csv(t, usecols=cols)
    df_list.append(df_phone)
df_truth = pd.concat(df_list, ignore_index=True)

train_base = pd.read_csv(path / "baseline_locations_train.csv")
all_df = df_truth.merge(train_base, how="inner", on=cols[:3], suffixes=("_truth", '_train_base'))

In [3]:
def get_groundtruth(path: Path) -> pd.DataFrame:
        output_df = pd.DataFrame()
        
        for path in glob(str(path / 'train/*/*/ground_truth.csv')):
            _df = pd.read_csv(path)
            output_df = pd.concat([output_df, _df])
        output_df = output_df.reset_index(drop=True)
        
        _columns = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']
        output_df[['t_'+col for col in _columns]] = output_df[_columns]
        output_df = output_df.drop(columns=_columns, axis=1)
        return output_df


In [4]:
train_base = train_base.merge(
    get_groundtruth(path),
    on=['collectionName', 'phoneName', 'millisSinceGpsEpoch']
)

# calc_haversine

In [5]:
def calc_haversine(lat1, lon1, lat2, lon2):
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    d = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(d**0.5)
    return dist

# Check Score

In [ ]:
def check_score(input_df: pd.DataFrame) -> pd.DataFrame:
    output_df = input_df.copy()
    
    output_df['meter'] = input_df.apply(
        lambda r: calc_haversine(
            r.latDeg, r.lngDeg, r.t_latDeg, r.t_lngDeg
        ),
        axis=1
    )

    meter_score = output_df['meter'].mean()
    print(f'error meter: {meter_score}')

    scores = []
    for phone in output_df['phone'].unique():
        _index = output_df['phone']==phone
        p_50 = np.percentile(output_df.loc[_index, 'meter'], 50)
        p_95 = np.percentile(output_df.loc[_index, 'meter'], 95)
        scores.append(p_50)
        scores.append(p_95)

    score = sum(scores) / len(scores)
    print(f'score: {score}')
    
    return output_df

In [ ]:
def mean_prediction_train(input_df):
    input_df["phone"] = input_df["collectionName"] + "_" + input_df["phoneName"]
    
    def add_distance_diff(df):
        df['latDeg_pre'] = df['latDeg'].shift(1)
        df['latDeg_pro'] = df['latDeg'].shift(-1)
        df['lngDeg_pre'] = df['lngDeg'].shift(1)
        df['lngDeg_pro'] = df['lngDeg'].shift(-1)
        df['phone_pre'] = df['phone'].shift(1)
        df['phone_pro'] = df['phone'].shift(-1)
        
        df['dist_pre'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_pre'], df['lngDeg_pre'])
        df['dist_pro'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_pro'], df['lngDeg_pro'])
        
        df.loc[df['phone']!=df['phone_pre'], ['latDeg_pre', 'lngDeg_pre', 'dist_pre']] = np.nan
        df.loc[df['phone']!=df['phone_pro'], ['latDeg_pro', 'lngDeg_pro', 'dist_pro']] = np.nan
        
        return df


    def make_lerp_data(input_df):
        org_colus = input_df.columns

        time_list = input_df[["collectionName", "millisSinceGpsEpoch"]].drop_duplicates()
        phone_list = input_df[["collectionName", "phoneName"]].drop_duplicates()
        tmp = time_list.merge(phone_list, on="collectionName", how="outer")

        output_df = tmp.merge(input_df, on=["collectionName", "millisSinceGpsEpoch", "phoneName"], how="left")
        output_df["phone"] = output_df["collectionName"] + "_" + output_df["phoneName"]
        output_df = output_df.sort_values(["phone", "millisSinceGpsEpoch"])

        output_df["latDeg_pre"] = output_df["latDeg"].shift(1)
        output_df["latDeg_pro"] = output_df["latDeg"].shift(-1)
        output_df["lngDeg_pre"] = output_df["lngDeg"].shift(1)
        output_df["lngDeg_pro"] = output_df["lngDeg"].shift(-1)
        output_df["phone_pre"] = output_df["phone"].shift(1)
        output_df["phone_pro"] = output_df["phone"].shift(-1)
        output_df["millisSinceGpsEpoch_pre"] = output_df["millisSinceGpsEpoch"].shift(1)
        output_df["millisSinceGpsEpoch_pro"] = output_df["millisSinceGpsEpoch"].shift(-1)

        output_df = output_df[(output_df["latDeg"].isnull())&(output_df["phone"] == output_df["phone_pre"])&
                            (output_df["phone"] == output_df["phone_pro"])].copy()

        #preとproの間を経過時間を考慮して算出
        output_df["latDeg"] = output_df["latDeg_pre"] + ((output_df["latDeg_pro"] - output_df["latDeg_pre"]) * 
                                                        ((output_df["millisSinceGpsEpoch"] - output_df["millisSinceGpsEpoch_pre"]) /
                                                        (output_df["millisSinceGpsEpoch_pro"] - output_df["millisSinceGpsEpoch_pre"])))
        output_df["lngDeg"] = output_df["lngDeg_pre"] + ((output_df["lngDeg_pro"] - output_df["lngDeg_pre"]) * 
                                                        ((output_df["millisSinceGpsEpoch"] - output_df["millisSinceGpsEpoch_pre"]) /
                                                        (output_df["millisSinceGpsEpoch_pro"] - output_df["millisSinceGpsEpoch_pre"])))
        
        output_df = output_df[~output_df['latDeg'].isnull()]

        return output_df[org_colus]

    
    def calc_mean_pred(input_df, lerp_df):
        input_df["phone"] = input_df["collectionName"] + "_" + input_df["phoneName"]
        add_lerp = pd.concat([input_df, lerp_df])
        mean_pred_result = add_lerp.groupby(["collectionName", "millisSinceGpsEpoch"])[["latDeg", "lngDeg"]].mean().reset_index()
        output_df = input_df[["collectionName", "phoneName", "millisSinceGpsEpoch"]].copy()
        output_df = output_df.merge(mean_pred_result[["collectionName", "millisSinceGpsEpoch", "latDeg", "lngDeg"]],
                                        on=["collectionName", "millisSinceGpsEpoch"], how="left")
        output_df["phone"] = output_df["collectionName"] + "_" + output_df["phoneName"]
        return output_df

    input_df_ = add_distance_diff(input_df)
    th = 50
    input_df_.loc[((input_df_['dist_pre'] > th) & (input_df_['dist_pro'] > th)), ['latDeg', 'lngDeg']] = np.nan

    #input_df_kalman = kalman_filter_train(input_df_)
    
    lerp = make_lerp_data(input_df_)
    mean_pred  = calc_mean_pred(input_df_, lerp)

    output_df = kalman_filter_train(mean_pred)
    output_df["t_latDeg"] = input_df["t_latDeg"]
    output_df["t_lngDeg"] = input_df["t_lngDeg"]
    
    return output_df

In [ ]:
def outlier_train(input_df):
    output_df = input_df.copy()
    output_df["phone"] = output_df["collectionName"] + "_" + output_df["phoneName"]

    output_df["dist_pre"] = 0
    output_df["dist_pro"] = 0

    output_df['latDeg_pre'] = output_df['latDeg'].shift(periods=1,fill_value=0)
    output_df['lngDeg_pre'] = output_df['lngDeg'].shift(periods=1,fill_value=0)
    output_df['latDeg_pro'] = output_df['latDeg'].shift(periods=-1,fill_value=0)
    output_df['lngDeg_pro'] = output_df['lngDeg'].shift(periods=-1,fill_value=0)
    output_df['dist_pre'] = calc_haversine(output_df.latDeg_pre, output_df.lngDeg_pre, output_df.latDeg, output_df.lngDeg)
    output_df['dist_pro'] = calc_haversine(output_df.latDeg, output_df.lngDeg, output_df.latDeg_pro, output_df.lngDeg_pro)

    list_phone = output_df['phone'].unique()
    for phone in list_phone:
        ind_s = output_df[train_base['phone'] == phone].index[0]
        ind_e = output_df[train_base['phone'] == phone].index[-1]
        output_df.loc[ind_s,'dist_pre'] = 0
        output_df.loc[ind_e,'dist_pro'] = 0

    pro_95 = output_df['dist_pro'].mean() + (output_df['dist_pro'].std() * 2)
    pre_95 = output_df['dist_pre'].mean() + (output_df['dist_pre'].std() * 2)
    ind = output_df[(output_df['dist_pro'] > pro_95)&(output_df['dist_pre'] > pre_95)][['dist_pre','dist_pro']].index

    for i in ind:
        output_df.loc[i,'latDeg'] = (output_df.loc[i-1,'latDeg'] + output_df.loc[i+1,'latDeg'])/2
        output_df.loc[i,'lngDeg'] = (output_df.loc[i-1,'lngDeg'] + output_df.loc[i+1,'lngDeg'])/2

    return output_df

In [ ]:
#position_shift
def position_shift_train(input_df):
    sub_cols = sub.columns

    train_p_s = pd.read_csv(path / "baseline_locations_train.csv")
    train_b = train_p_s[sub_cols]
    test_b = test_base[sub_cols]

    msge = "millisSinceGpsEpoch"

    testdir = path / 'test'
    traindir = path / 'train'

    g_t = pd.DataFrame()
    for d in os.listdir(traindir):
        for p in os.listdir(traindir/d):
            g_t = g_t.append(pd.read_csv(traindir/d/p/'ground_truth.csv'))

    g_t["phone"] = g_t["collectionName"] + "_" + g_t["phoneName"]
    g_t_sub_cols = g_t[sub_cols]


    def compute_dist(fname_1, fname_2=g_t):
        oof = fname_1
        g_t = fname_2
        df = oof.merge(g_t, on=["phone", "millisSinceGpsEpoch"])
        dist_oof = calc_haversine(df.latDeg_x, df.lngDeg_x, df.latDeg_y, df.lngDeg_y)
        scores = pd.DataFrame({"phone":df.phone, "dist":dist_oof})
        scores_grp = scores.groupby("phone")
        d_50 = scores_grp.quantile(.50).reset_index()
        d_50.columns = ["phone", "q_50"]
        d_95 = scores_grp.quantile(.95).reset_index()
        d_95.columns = ["phone", "q_95"]
        return (scores_grp.quantile(.50).mean() + scores_grp.quantile(.95).mean()) / 2, d_50.merge(d_95)


    def WGS84_to_ECEF(lat, lng, alt):
        rad_lat = lat * (np.pi / 180.0)
        rad_lng = lng * (np.pi / 180.0)
        a = 6378137.0
        finv = 298.257223563
        f = 1 / finv   
        e2 = 1 - (1 - f) * (1 - f)    
        N = a / np.sqrt(1 - e2 * np.sin(rad_lat) * np.sin(rad_lat))
        x = (N + alt) * np.cos(rad_lat) * np.cos(rad_lng)
        y = (N + alt) * np.cos(rad_lat) * np.sin(rad_lng)
        z = (N * (1 - e2) + alt)        * np.sin(rad_lat)
        return x, y, z    


    transformer = pyproj.Transformer.from_crs(
        {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
        {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},)


    def ECEF_to_WGS84(x, y, z):
        lng, lat, alt = transformer.transform(x, y, z, radians=False)
        return lng, lat, alt


    def position_shift_(input_df, a):
        output_df = input_df.copy()
        output_df["heightAboveWgs84EllipsoidM"] = 63.5
        output_df["x"], output_df["y"], output_df["z"] = zip(*output_df.apply(lambda x: WGS84_to_ECEF(x.latDeg, x.lngDeg, 
                                                                                                    x.heightAboveWgs84EllipsoidM), axis=1))
        output_df.sort_values(["phone", msge], inplace=True)
        for fi in ["x", "y", "z"]:
            output_df[[fi + "_p"]] = output_df[fi].shift().where(output_df["phone"].eq(output_df["phone"].shift()))
            output_df[[fi + "_diff"]] = output_df[fi] - output_df[fi + "_p"]

        output_df[["dist"]] = np.sqrt(output_df["x_diff"]**2 + output_df["y_diff"]**2 + output_df["z_diff"]**2)
        for fi in ["x", "y", "z"]:
            output_df[[fi + "_new"]] = output_df[fi + "_p"] + output_df[fi + "_diff"] * (1 - a/output_df["dist"])
        lng, lat, alt = ECEF_to_WGS84(output_df["x_new"].values, output_df["y_new"].values, output_df["z_new"].values)

        lat[np.isnan(lat)] = output_df.loc[np.isnan(lat), "latDeg"]
        lng[np.isnan(lng)] = output_df.loc[np.isnan(lng), "lngDeg"]
        output_df["latDeg"] = lat
        output_df["lngDeg"] = lng

        output_df.sort_values(["phone", msge], inplace=True)

        return output_df[sub_cols]


    def objective(trial):
        a = trial.suggest_uniform("a", -1, 1)
        score, scores = compute_dist(position_shift_(train_b, a), g_t)
        return score


    study = optuna.create_study()
    study.optimize(objective, n_trials=30)

    output_df = position_shift_(input_df, a=study.best_params["a"])

    output_df["phoneName"] = input_df["phoneName"]
    output_df["collectionName"] = input_df["collectionName"]
    output_df["millisSinceGpsEpoch"] = input_df["millisSinceGpsEpoch"]
    output_df["t_latDeg"] = input_df["t_latDeg"]
    output_df["t_lngDeg"] = input_df["t_lngDeg"]

    return output_df

In [ ]:
#remove_device
def remove_device_train(input_df):

    def get_removedevice(input_df: pd.DataFrame, divece: str) -> pd.DataFrame:
        input_df['index'] = input_df.index
        input_df = input_df.sort_values('millisSinceGpsEpoch')
        input_df.index = input_df['millisSinceGpsEpoch'].values

        output_df = pd.DataFrame() 
        for _, subdf in input_df.groupby('collectionName'):

            phones = subdf['phoneName'].unique()

            if (len(phones) == 1) or (not divece in phones):
                output_df = pd.concat([output_df, subdf])
                continue

            origin_df = subdf.copy()
            
            _index = subdf['phoneName']==divece
            subdf.loc[_index, 'latDeg'] = np.nan
            subdf.loc[_index, 'lngDeg'] = np.nan
            subdf = subdf.interpolate(method='index', limit_area='inside')

            _index = subdf['latDeg'].isnull()
            subdf.loc[_index, 'latDeg'] = origin_df.loc[_index, 'latDeg'].values
            subdf.loc[_index, 'lngDeg'] = origin_df.loc[_index, 'lngDeg'].values

            output_df = pd.concat([output_df, subdf])

        output_df.index = output_df['index'].values
        output_df = output_df.sort_index()

        del output_df['index']
        
        return output_df

    output_df = get_removedevice(input_df, 'SamsungS20Ultra')

    return output_df

In [ ]:
def kalman_filter_train(input_df):
    T = 1.0
    state_transition = np.array([[1, 0, T, 0, 0.5 * T ** 2, 0], [0, 1, 0, T, 0, 0.5 * T ** 2], [0, 0, 1, 0, T, 0],
                                [0, 0, 0, 1, 0, T], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])
    process_noise = np.diag([1e-5, 1e-5, 5e-6, 5e-6, 1e-6, 1e-6]) + np.ones((6, 6)) * 1e-9
    observation_model = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])
    observation_noise = np.diag([6e-5, 6e-5]) + np.ones((2, 2)) * 1e-9

    kf = simdkalman.KalmanFilter(
            state_transition = state_transition,
            process_noise = process_noise,
            observation_model = observation_model,
            observation_noise = observation_noise)
    
    def apply_kf_smoothing(input_df, kf_=kf):
        output_df = input_df.copy()
        unique_paths = output_df[['collectionName', 'phoneName']].drop_duplicates().to_numpy()
        for collection, phone in tqdm(unique_paths):
            cond = np.logical_and(output_df['collectionName'] == collection, output_df['phoneName'] == phone)
            data = output_df[cond][['latDeg', 'lngDeg']].to_numpy()
            data = data.reshape(1, len(data), 2)
            smoothed = kf_.smooth(data)
            output_df.loc[cond, 'latDeg'] = smoothed.states.mean[0, :, 0]
            output_df.loc[cond, 'lngDeg'] = smoothed.states.mean[0, :, 1]
        return output_df

    output_df = apply_kf_smoothing(input_df)

    return output_df


In [ ]:
#check_score
CV_1 = outlier_train(train_base)
print("----------------------")
check_score(CV_1)
print("----------------------")
CV_2 = mean_prediction_train(CV_1)
print("----------------------")
check_score(CV_2)
print("----------------------")
#CV_3 = kalman_filter_train(CV_2)
print("----------------------")
#check_score(CV_3)
print("----------------------")
CV_4 = remove_device_train(CV_2)
print("----------------------")
check_score(CV_4)
print("----------------------")
CV_5 = position_shift_train(CV_4)
print("----------------------")
check_score(CV_5)
print("----------------------")

----------------------
error meter: 3.596487013261986
score: nan
----------------------



----------------------
error meter: 2.8954066169743142
score: 4.09097421928831
----------------------
----------------------
----------------------
----------------------
error meter: 2.8094801726186986
score: 3.946250165667493
----------------------


[I 2021-07-03 22:53:11,645] A new study created in memory with name: no-name-20c33583-1ad4-4d29-9864-7d6cb35cd06b
[I 2021-07-03 22:53:17,971] Trial 0 finished with value: 5.3061727145650766 and parameters: {'a': -0.06227799659255129}. Best is trial 0 with value: 5.3061727145650766.
[I 2021-07-03 22:53:24,405] Trial 1 finished with value: 5.319289475674226 and parameters: {'a': -0.10503451339528258}. Best is trial 0 with value: 5.3061727145650766.
[I 2021-07-03 22:53:30,854] Trial 2 finished with value: 5.245855579440381 and parameters: {'a': 0.1494020977767745}. Best is trial 2 with value: 5.245855579440381.
[I 2021-07-03 22:53:37,256] Trial 3 finished with value: 5.624256551750639 and parameters: {'a': -0.7753024782390656}. Best is trial 2 with value: 5.245855579440381.
[I 2021-07-03 22:53:43,663] Trial 4 finished with value: 5.567605817861847 and parameters: {'a': -0.669413440558964}. Best is trial 2 with value: 5.245855579440381.
[I 2021-07-03 22:53:50,182] Trial 5 finished with val

----------------------
error meter: 2.773344222849986
score: 3.9073382758386455
----------------------


# Mean Prediction

In [12]:
def mean_prediction(input_df):
    def add_distance_diff(input_df):
        output_df = input_df.copy()
        output_df['latDeg_pre'] = output_df['latDeg'].shift(1)
        output_df['latDeg_pro'] = output_df['latDeg'].shift(-1)
        output_df['lngDeg_pre'] = output_df['lngDeg'].shift(1)
        output_df['lngDeg_pro'] = output_df['lngDeg'].shift(-1)
        output_df['phone_pre'] = output_df['phone'].shift(1)
        output_df['phone_pro'] = output_df['phone'].shift(-1)
        
        output_df['dist_pre'] = calc_haversine(output_df['latDeg'], output_df['lngDeg'], output_df['latDeg_pre'], output_df['lngDeg_pre'])
        output_df['dist_pro'] = calc_haversine(output_df['latDeg'], output_df['lngDeg'], output_df['latDeg_pro'], output_df['lngDeg_pro'])
        
        output_df.loc[output_df['phone']!=output_df['phone_pre'], ['latDeg_pre', 'lngDeg_pre', 'dist_pre']] = np.nan
        output_df.loc[output_df['phone']!=output_df['phone_pro'], ['latDeg_pro', 'lngDeg_pro', 'dist_pro']] = np.nan
        
        return output_df

    input_df_ = add_distance_diff(input_df)
    th = 50
    input_df_.loc[((input_df_['dist_pre'] > th) & (input_df_['dist_pro'] > th)), ['latDeg', 'lngDeg']] = np.nan

    def make_lerp_data(input_df):
        org_colus = input_df.columns

        time_list = input_df[["collectionName", "millisSinceGpsEpoch"]].drop_duplicates()
        phone_list = input_df[["collectionName", "phoneName"]].drop_duplicates()
        tmp = time_list.merge(phone_list, on="collectionName", how="outer")

        output_df = tmp.merge(input_df, on=["collectionName", "millisSinceGpsEpoch", "phoneName"], how="left")
        output_df["phone"] = output_df["collectionName"] + "_" + output_df["phoneName"]
        output_df = output_df.sort_values(["phone", "millisSinceGpsEpoch"])

        output_df["latDeg_pre"] = output_df["latDeg"].shift(1)
        output_df["latDeg_pro"] = output_df["latDeg"].shift(-1)
        output_df["lngDeg_pre"] = output_df["lngDeg"].shift(1)
        output_df["lngDeg_pro"] = output_df["lngDeg"].shift(-1)
        output_df["phone_pre"] = output_df["phone"].shift(1)
        output_df["phone_pro"] = output_df["phone"].shift(-1)
        output_df["millisSinceGpsEpoch_pre"] = output_df["millisSinceGpsEpoch"].shift(1)
        output_df["millisSinceGpsEpoch_pro"] = output_df["millisSinceGpsEpoch"].shift(-1)

        output_df = output_df[(output_df["latDeg"].isnull())&(output_df["phone"] == output_df["phone_pre"])&
                            (output_df["phone"] == output_df["phone_pro"])].copy()

        output_df["latDeg"] = output_df["latDeg_pre"] + ((output_df["latDeg_pro"] - output_df["latDeg_pre"]) * 
                                                        ((output_df["millisSinceGpsEpoch"] - output_df["millisSinceGpsEpoch_pre"]) /
                                                        (output_df["millisSinceGpsEpoch_pro"] - output_df["millisSinceGpsEpoch_pre"])))
        output_df["lngDeg"] = output_df["lngDeg_pre"] + ((output_df["lngDeg_pro"] - output_df["lngDeg_pre"]) * 
                                                        ((output_df["millisSinceGpsEpoch"] - output_df["millisSinceGpsEpoch_pre"]) /
                                                        (output_df["millisSinceGpsEpoch_pro"] - output_df["millisSinceGpsEpoch_pre"])))
        
        output_df = output_df[~output_df['latDeg'].isnull()]

        return output_df[org_colus]

    
    def calc_mean_pred(input_df, lerp_df):
        input_df["phone"] = input_df["collectionName"] + "_" + input_df["phoneName"]
        add_lerp = pd.concat([input_df, lerp_df])
        mean_pred_result = add_lerp.groupby(["collectionName", "millisSinceGpsEpoch"])[["latDeg", "lngDeg"]].mean().reset_index()
        output_df = input_df[["collectionName", "phoneName", "millisSinceGpsEpoch"]].copy()
        output_df = output_df.merge(mean_pred_result[["collectionName", "millisSinceGpsEpoch", "latDeg", "lngDeg"]],
                                        on=["collectionName", "millisSinceGpsEpoch"], how="left")
        output_df["phone"] = output_df["collectionName"] + "_" + output_df["phoneName"]
        return output_df

    test_lerp = make_lerp_data(input_df_)
    test_mean_pred  = calc_mean_pred(input_df_, test_lerp)

    output_df = kalman_filter(test_mean_pred)    
    
    return output_df

# Outlier 

In [7]:
def outlier(input_df):
    output_df = input_df

    output_df["dist_pre"] = 0
    output_df["dist_pro"] = 0

    output_df['latDeg_pre'] = output_df['latDeg'].shift(periods=1,fill_value=0)
    output_df['lngDeg_pre'] = output_df['lngDeg'].shift(periods=1,fill_value=0)
    output_df['latDeg_pro'] = output_df['latDeg'].shift(periods=-1,fill_value=0)
    output_df['lngDeg_pro'] = output_df['lngDeg'].shift(periods=-1,fill_value=0)
    output_df['dist_pre'] = calc_haversine(output_df.latDeg_pre, output_df.lngDeg_pre, output_df.latDeg, output_df.lngDeg)
    output_df['dist_pro'] = calc_haversine(output_df.latDeg, output_df.lngDeg, output_df.latDeg_pro, output_df.lngDeg_pro)

    list_phone = output_df['phone'].unique()
    for phone in list_phone:
        ind_s = output_df[test_base['phone'] == phone].index[0]
        ind_e = output_df[test_base['phone'] == phone].index[-1]
        output_df.loc[ind_s,'dist_pre'] = 0
        output_df.loc[ind_e,'dist_pro'] = 0

    pro_95 = output_df['dist_pro'].mean() + (output_df['dist_pro'].std() * 2)
    pre_95 = output_df['dist_pre'].mean() + (output_df['dist_pre'].std() * 2)
    ind = output_df[(output_df['dist_pro'] > pro_95)&(output_df['dist_pre'] > pre_95)][['dist_pre','dist_pro']].index

    for i in ind:
        output_df.loc[i,'latDeg'] = (output_df.loc[i-1,'latDeg'] + output_df.loc[i+1,'latDeg'])/2
        output_df.loc[i,'lngDeg'] = (output_df.loc[i-1,'lngDeg'] + output_df.loc[i+1,'lngDeg'])/2

    output_df["phoneName"] = input_df["phoneName"]
    output_df["collectionName"] = input_df["collectionName"]
    output_df["millisSinceGpsEpoch"] = input_df["millisSinceGpsEpoch"]

    return output_df

# Position Shift

In [8]:
def position_shift(input_df):
    sub_cols = sub.columns

    train_p_s = pd.read_csv(path / "baseline_locations_train.csv")
    train_b = train_p_s[sub_cols]
    test_b = test_base[sub_cols]

    msge = "millisSinceGpsEpoch"

    testdir = path / 'test'
    traindir = path / 'train'

    g_t = pd.DataFrame()
    for d in os.listdir(traindir):
        for p in os.listdir(traindir/d):
            g_t = g_t.append(pd.read_csv(traindir/d/p/'ground_truth.csv'))

    g_t["phone"] = g_t["collectionName"] + "_" + g_t["phoneName"]
    g_t_sub_cols = g_t[sub_cols]


    def compute_dist(fname_1, fname_2=g_t):
        oof = fname_1
        g_t = fname_2
        df = oof.merge(g_t, on=["phone", "millisSinceGpsEpoch"])
        dist_oof = calc_haversine(df.latDeg_x, df.lngDeg_x, df.latDeg_y, df.lngDeg_y)
        scores = pd.DataFrame({"phone":df.phone, "dist":dist_oof})
        scores_grp = scores.groupby("phone")
        d_50 = scores_grp.quantile(.50).reset_index()
        d_50.columns = ["phone", "q_50"]
        d_95 = scores_grp.quantile(.95).reset_index()
        d_95.columns = ["phone", "q_95"]
        return (scores_grp.quantile(.50).mean() + scores_grp.quantile(.95).mean()) / 2, d_50.merge(d_95)


    def WGS84_to_ECEF(lat, lng, alt):
        rad_lat = lat * (np.pi / 180.0)
        rad_lng = lng * (np.pi / 180.0)
        a = 6378137.0
        finv = 298.257223563
        f = 1 / finv   
        e2 = 1 - (1 - f) * (1 - f)    
        N = a / np.sqrt(1 - e2 * np.sin(rad_lat) * np.sin(rad_lat))
        x = (N + alt) * np.cos(rad_lat) * np.cos(rad_lng)
        y = (N + alt) * np.cos(rad_lat) * np.sin(rad_lng)
        z = (N * (1 - e2) + alt)        * np.sin(rad_lat)
        return x, y, z    


    transformer = pyproj.Transformer.from_crs(
        {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
        {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},)


    def ECEF_to_WGS84(x, y, z):
        lng, lat, alt = transformer.transform(x, y, z, radians=False)
        return lng, lat, alt


    def position_shift_(input_df, a):
        output_df = input_df.copy()
        output_df["heightAboveWgs84EllipsoidM"] = 63.5
        output_df["x"], output_df["y"], output_df["z"] = zip(*output_df.apply(lambda x: WGS84_to_ECEF(x.latDeg, x.lngDeg, 
                                                                                                    x.heightAboveWgs84EllipsoidM), axis=1))
        output_df.sort_values(["phone", msge], inplace=True)
        for fi in ["x", "y", "z"]:
            output_df[[fi + "_p"]] = output_df[fi].shift().where(output_df["phone"].eq(output_df["phone"].shift()))
            output_df[[fi + "_diff"]] = output_df[fi] - output_df[fi + "_p"]

        output_df[["dist"]] = np.sqrt(output_df["x_diff"]**2 + output_df["y_diff"]**2 + output_df["z_diff"]**2)
        for fi in ["x", "y", "z"]:
            output_df[[fi + "_new"]] = output_df[fi + "_p"] + output_df[fi + "_diff"] * (1 - a/output_df["dist"])
        lng, lat, alt = ECEF_to_WGS84(output_df["x_new"].values, output_df["y_new"].values, output_df["z_new"].values)

        lat[np.isnan(lat)] = output_df.loc[np.isnan(lat), "latDeg"]
        lng[np.isnan(lng)] = output_df.loc[np.isnan(lng), "lngDeg"]
        output_df["latDeg"] = lat
        output_df["lngDeg"] = lng

        output_df.sort_values(["phone", msge], inplace=True)

        return output_df[sub_cols]


    def objective(trial):
        a = trial.suggest_uniform("a", -1, 1)
        score, scores = compute_dist(position_shift_(train_b, a), g_t)
        return score


    study = optuna.create_study()
    study.optimize(objective, n_trials=30)

    output_df = position_shift_(input_df, a=study.best_params["a"])

    output_df["phoneName"] = input_df["phoneName"]
    output_df["collectionName"] = input_df["collectionName"]
    output_df["millisSinceGpsEpoch"] = input_df["millisSinceGpsEpoch"]
    #output_df["t_latDeg"] = input_df["t_latDeg"]
    #output_df["t_lngDeg"] = input_df["t_lngDeg"]

    return output_df

# Remove Device

In [9]:
def remove_device(input_df):

    def get_removedevice(input_df: pd.DataFrame, divece: str) -> pd.DataFrame:
        input_df['index'] = input_df.index
        input_df = input_df.sort_values('millisSinceGpsEpoch')
        input_df.index = input_df['millisSinceGpsEpoch'].values

        output_df = pd.DataFrame() 
        for _, subdf in input_df.groupby('collectionName'):

            phones = subdf['phoneName'].unique()

            if (len(phones) == 1) or (not divece in phones):
                output_df = pd.concat([output_df, subdf])
                continue

            origin_df = subdf.copy()
            
            _index = subdf['phoneName']==divece
            subdf.loc[_index, 'latDeg'] = np.nan
            subdf.loc[_index, 'lngDeg'] = np.nan
            subdf = subdf.interpolate(method='index', limit_area='inside')

            _index = subdf['latDeg'].isnull()
            subdf.loc[_index, 'latDeg'] = origin_df.loc[_index, 'latDeg'].values
            subdf.loc[_index, 'lngDeg'] = origin_df.loc[_index, 'lngDeg'].values

            output_df = pd.concat([output_df, subdf])

        output_df.index = output_df['index'].values
        output_df = output_df.sort_index()

        del output_df['index']
        
        return output_df

    output_df = get_removedevice(input_df, 'SamsungS20Ultra')

    return output_df

# Kalman Filter

In [10]:
def kalman_filter(input_df):

    T = 1.0
    state_transition = np.array([[1, 0, T, 0, 0.5 * T ** 2, 0], [0, 1, 0, T, 0, 0.5 * T ** 2], [0, 0, 1, 0, T, 0],
                                [0, 0, 0, 1, 0, T], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])
    process_noise = np.diag([1e-5, 1e-5, 5e-6, 5e-6, 1e-6, 1e-6]) + np.ones((6, 6)) * 1e-9
    observation_model = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])
    observation_noise = np.diag([6e-5, 6e-5]) + np.ones((2, 2)) * 1e-9

    kf = simdkalman.KalmanFilter(
            state_transition = state_transition,
            process_noise = process_noise,
            observation_model = observation_model,
            observation_noise = observation_noise)
    
    def apply_kf_smoothing(df, kf_=kf):
        unique_paths = df[['collectionName', 'phoneName']].drop_duplicates().to_numpy()
        for collection, phone in tqdm(unique_paths):
            cond = np.logical_and(df['collectionName'] == collection, df['phoneName'] == phone)
            data = df[cond][['latDeg', 'lngDeg']].to_numpy()
            data = data.reshape(1, len(data), 2)
            smoothed = kf_.smooth(data)
            df.loc[cond, 'latDeg'] = smoothed.states.mean[0, :, 0]
            df.loc[cond, 'lngDeg'] = smoothed.states.mean[0, :, 1]
        return df

    kf_smoothed_baseline = apply_kf_smoothing(input_df)
    output_df = sub.assign(latDeg = kf_smoothed_baseline.latDeg, lngDeg = kf_smoothed_baseline.lngDeg)
    output_df["phoneName"] = input_df["phoneName"]
    output_df["collectionName"] = input_df["collectionName"]
    output_df["millisSinceGpsEpoch"] = input_df["millisSinceGpsEpoch"]
    #output_df["t_latDeg"] = input_df["t_latDeg"]
    #output_df["t_lngDeg"] = input_df["t_lngDeg"]
    
    return output_df

# SUB

In [17]:
submission_1 = outlier(test_base)
submission_2 = mean_prediction(submission_1)
submission_3 = remove_device(submission_2)
submission_4 = position_shift(submission_3)
#submission_5 = kalman_filter(submission_4)

[I 2021-07-15 10:11:01,395] A new study created in memory with name: no-name-1ae228e1-0369-41dc-a953-40ed5db0efba
[I 2021-07-15 10:11:07,807] Trial 0 finished with value: 5.258798208566426 and parameters: {'a': 0.0970091062978764}. Best is trial 0 with value: 5.258798208566426.
[I 2021-07-15 10:11:14,409] Trial 1 finished with value: 5.1949039772313 and parameters: {'a': 0.45503549538648413}. Best is trial 1 with value: 5.1949039772313.
[I 2021-07-15 10:11:20,838] Trial 2 finished with value: 5.207930945648469 and parameters: {'a': 0.34089283716619745}. Best is trial 1 with value: 5.1949039772313.
[I 2021-07-15 10:11:27,482] Trial 3 finished with value: 5.598078701500132 and parameters: {'a': -0.7261421267892376}. Best is trial 1 with value: 5.1949039772313.
[I 2021-07-15 10:11:33,886] Trial 4 finished with value: 5.190010382723863 and parameters: {'a': 0.7103435473277953}. Best is trial 4 with value: 5.190010382723863.
[I 2021-07-15 10:11:40,471] Trial 5 finished with value: 5.2021602

In [18]:
submission_4.isna().sum()

phone                  0
millisSinceGpsEpoch    0
latDeg                 0
lngDeg                 0
phoneName              0
collectionName         0
dtype: int64

In [19]:
submission_4[sub.columns].to_csv('submission_34.csv', index=False)